In [75]:
import pandas as pd
import os
import yfinance as yf
from datetime import timedelta
import numpy as np

# --- Macro Loader ---
def load_macro_data():
    macro_path = os.path.join("..", "Economic Regime Forecaster", "Predicted Macro Regimes.csv")
    df = pd.read_csv(macro_path, parse_dates=["Date"])
    df = df[["Date", "Macro Regime"]].rename(columns={"Macro Regime": "Macro"})
    return df

# --- Volatility Loader ---
def add_vol_column(macro_df, ticker):
    vol_path = os.path.join("..", "Volatility Regime Forecaster", "Predicted Regimes", f"PredictedVol_{ticker}.csv")
    vol_df = pd.read_csv(vol_path, parse_dates=["Date"])
    vol_df = vol_df[["Date", "Vol Regime"]].rename(columns={"Vol Regime": "Vol"})
    df = pd.merge(macro_df, vol_df, on="Date", how="left")
    first_valid_index = df[['Macro', 'Vol']].dropna().index.min()
    df = df.loc[first_valid_index:].reset_index(drop=True)
    df["Vol"] = df["Vol"].bfill()
    return df

# --- Security Loader (No Threshold) ---
def add_price_column(df, ticker, start_date=None):
    if start_date is None:
        start_date = df["Date"].min()

    price_data = yf.download(ticker + "=F", start=start_date, progress=False)

    # --- Flatten multi-level structure if needed ---
    if isinstance(price_data.columns, pd.MultiIndex):
        price_data.columns = price_data.columns.get_level_values(0)

    # --- Reset index name (IMPORTANT) ---
    price_data.columns.name = None

    # --- Reset index to make Date a column ---
    price_data = price_data.reset_index()

    # --- Drop 'Price' column if it exists ---
    if "Price" in price_data.columns:
        price_data = price_data.drop(columns=["Price"])

    # --- Check if 'Close' exists, otherwise fallback to 'Adj Close' ---
    if "Close" in price_data.columns:
        price_col = "Close"
    elif "Adj Close" in price_data.columns:
        price_col = "Adj Close"
    else:
        print(f"⚠️  Warning: Neither 'Close' nor 'Adj Close' found for {ticker}. Skipping this ticker.")
        df["Price"] = np.nan
        return df

    # --- Select Date + price column ---
    price_data = price_data[["Date", price_col]].rename(columns={price_col: "Price"})

    df = pd.merge(df, price_data, on="Date", how="left")
    first_valid_index = df[['Macro', 'Vol', 'Price']].dropna().index.min()
    df = df.loc[first_valid_index:].reset_index(drop=True)
    df["Price"] = df["Price"].bfill()

    return df



# --- Price Data Loader ---
def add_price_column(df, ticker, start_date=None):
    if start_date is None:
        start_date = df["Date"].min()

    price_data = yf.download(ticker + "=F", start=start_date, progress=False)

    # --- If MultiIndex, remove second level (ticker) ---
    if isinstance(price_data.columns, pd.MultiIndex):
        price_data.columns = price_data.columns.get_level_values(0)

    # --- Reset index to get 'Date' as column ---
    price_data = price_data.reset_index()

    # --- Check if 'Close' exists ---
    if "Close" not in price_data.columns:
        print(f"⚠️  Warning: No 'Close' price found for {ticker}. Skipping this ticker.")
        df["Price"] = np.nan
        return df

    price_data = price_data[["Date", "Close"]].rename(columns={"Close": "Price"})

    df = pd.merge(df, price_data, on="Date", how="left")
    first_valid_index = df[['Macro', 'Vol', 'Price']].dropna().index.min()
    df = df.loc[first_valid_index:].reset_index(drop=True)
    df["Price"] = df["Price"].bfill()

    return df




# --- Enrichment with OHLCV ---
def enrich_with_ohlcv(ticker, current_df):
    start_date = current_df["Date"].min() - timedelta(days=90)

    ohlcv = yf.download(ticker + "=F", start=start_date, progress=False)

    # --- Flatten MultiIndex if needed ---
    if isinstance(ohlcv.columns, pd.MultiIndex):
        ohlcv.columns = ohlcv.columns.get_level_values(0)

    # --- Reset column names ---
    ohlcv.columns.name = None

    # --- Reset index to make Date a normal column ---
    ohlcv = ohlcv.reset_index()

    # --- Drop 'Price' if exists (leftover junk) ---
    if "Price" in ohlcv.columns:
        ohlcv = ohlcv.drop(columns=["Price"])

    # --- Only keep necessary columns ---
    ohlcv = ohlcv[["Date", "Open", "High", "Low", "Close", "Volume"]]

    return ohlcv


# --- Compute AVWAP and Pivot Points ---
def compute_avwap_and_pivots(current_df, ohlcv_df):
    df = pd.merge(current_df, ohlcv_df, on="Date", how="left")
    df["30d_high"] = df["Close"].rolling(window=30, min_periods=1).max()
    df["AVWAP"] = float("nan")
    cum_pv, cum_v, anchor_index = 0.0, 0.0, None

    for i in range(len(df)):
        if df["Close"].iloc[i] == df["30d_high"].iloc[i]:
            anchor_index = i
            cum_pv, cum_v = 0.0, 0.0
        if anchor_index is not None:
            price = df["Close"].iloc[i]
            volume = df["Volume"].iloc[i]
            cum_pv += price * volume
            cum_v += volume
            df.at[i, "AVWAP"] = cum_pv / cum_v if cum_v != 0 else price

    df["Base"] = (df["High"] + df["Low"] + df["Close"]) / 3
    df["R1"] = 2 * df["Base"] - df["Low"]
    df["S1"] = 2 * df["Base"] - df["High"]
    df["R2"] = df["Base"] + (df["High"] - df["Low"])
    df["S2"] = df["Base"] - (df["High"] - df["Low"])

    start_date = current_df["Date"].min()
    df = df[df["Date"] >= start_date].reset_index(drop=True)

    # Final clean-up: enforce consistent column order
    final_columns = ["Date", "Macro", "Vol", "Security", "Security Proba", "Price", "AVWAP", "Base", "R1", "R2", "S1", "S2"]
    df = df[final_columns]

    # --- Convert everything except 'Date' to float32 ---
    for col in df.columns:
        if col != "Date":
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)

    return df


# --- Master Function to Build and Save All ---
def build_and_save_rl_datasets(ticker_list):
    macro_df = load_macro_data()

    for ticker in ticker_list:
        print(f"Processing {ticker}...")

        df = add_vol_column(macro_df, ticker)
        df = add_security_column(df, ticker)
        df = add_price_column(df, ticker)
        ohlcv_df = enrich_with_ohlcv(ticker, df)
        final_df = compute_avwap_and_pivots(df, ohlcv_df)

        output_name = f"RL - {ticker}.csv"
        final_df.to_csv(output_name, index=False, float_format="%.5f")
        print(f"Saved {output_name}")


In [76]:
tickers = ["ES", "CL", "ZN"]  # or whatever you want
build_and_save_rl_datasets(tickers)

Processing ES...
Saved RL - ES.csv
Processing CL...
Saved RL - CL.csv
Processing ZN...
Saved RL - ZN.csv
